In [1]:
import pandas as pd 
import numpy as np
import lightgbm as lgb
import sklearn

In [9]:
train = pd.read_csv('input/FEd_train.csv')
tr = train.copy()

In [10]:
tt = pd.read_csv('input/train.csv',dtype={'WindSpeed': 'object'})

In [15]:
train = tt.copy()

In [11]:
print(len(train.columns), len(tt.columns))

57 49


In [14]:
train.columns

Index(['X', 'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir', 'NflId',
       'JerseyNumber', 'Season', 'YardLine', 'Quarter', 'GameClock',
       'PossessionTeam', 'Down', 'Distance', 'FieldPosition',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'NflIdRusher',
       'OffenseFormation', 'DefendersInTheBox', 'Yards', 'PlayerHeight',
       'PlayerWeight', 'PlayerBirthDate', 'Position', 'Week', 'StadiumType',
       'Turf', 'GameWeather', 'Temperature', 'Humidity', 'WindSpeed',
       'TotalScore', 'CumGameClock', 'VsTeam', 'HomeTeam', 'IsOffense',
       'TimeDelta', 'HomeField', 'Field_eq_Possession', 'OFF_RB', 'OFF_DB',
       'OFF_OL', 'OFF_WR', 'OFF_DL', 'OFF_TE', 'OFF_QB', 'OFF_LB', 'DEF_OL',
       'DEF_DL', 'DEF_DB', 'DEF_LB', 'WinRatio', 'VsWinRatio', 'Conf'],
      dtype='object')

In [13]:
categorical = [
    'NflId', 'JerseyNumber', 'Season', 'Quarter', 'PossessionTeam', 'FieldPosition',
    'NflIdRusher', 'OffenseFormation',  'Position', 'Week',
    'StadiumType', 'Turf', 'VsTeam', 'HomeTeam', 'IsOffense', 'HomeField', 
    'Field_eq_Possession', 'Conf'
]

numerical = []
for i in tr.columns:
    if i not in categorical:
        numerical.append(i)
        
ctd = [
    'GameId', 'PlayId', 'OffensePersonnel', 'DefensePersonnel',
    'PlayDirection', 'PlayerCollegeName', 'Stadium', 'WindDirection', 'Location',
    'DisplayName', 'Team', 'TimeHandoff', 'TimeSnap'
    
]

In [31]:
winratio17_18 = {
    'KC': 0.625,
    'NE': 0.813,
    'NYJ': 0.313,
    'BUF': 0.563,
    'ATL': 0.625,
    'CHI': 0.313,
    'BLT': 0.563,
    'CIN': 0.438,
    'PIT': 0.813,
    'CLV': 0,
    'ARZ': 0.5,
    'DET': 0.563,
    'JAX': 0.625,
    'HST': 0.25,
    'OAK': 0.375,
    'TEN': 0.563,
    'PHI': 0.813,
    'WAS': 0.438,
    'IND': 0.25,
    'LA': 0.688,
    'SEA': 0.563,
    'GB': 0.438,
    'CAR': 0.688,
    'SF': 0.375,
    'NYG': 0.188,
    'DAL': 0.563,
    'NO': 0.688,
    'MIN': 0.813,
    'LAC': 0.563,
    'DEN': 0.313,
    'TB': 0.313,
    'MIA': 0.375
}

winratio18_19 = {
    'KC': 0.75,
    'NE':0.688,
    'NYJ': 0.25,
    'BUF': 0.375,
    'ATL': 0.438,
    'CHI': 0.75,
    'BLT': 0.625,
    'CIN': 0.375,
    'PIT': 0.594,
    'CLV': 0.469,
    'ARZ': 0.188,
    'DET': 0.375,
    'JAX': 0.313,
    'HST': 0.688,
    'OAK': 0.25,
    'TEN': 0.563,
    'PHI': 0.563,
    'WAS': 0.438,
    'IND': 0.625,
    'LA': 0.813,
    'SEA': 0.625,
    'GB': 0.406,
    'CAR': 0.438,
    'SF': 0.25,
    'NYG': 0.313,
    'DAL': 0.625,
    'NO': 0.813,
    'MIN': 0.531,
    'LAC': 0.75,
    'DEN': 0.375,
    'TB': 0.313,
    'MIA': 0.438
}

winratio19_20 = {
    'KC': 0.714,
    'NE':1,
    'NYJ': 0.2,
    'BUF': 0.833,
    'ATL': 0.143,
    'CHI': 0.5,
    'BLT': 0.714,
    'CIN': 0,
    'PIT': 0.333,
    'CLV': 0.333,
    'ARZ': 0.5,
    'DET': 0.417,
    'JAX': 0.429,
    'HST': 0.571,
    'OAK': 0.5,
    'TEN': 0.429,
    'PHI': 0.429,
    'WAS': 0.143,
    'IND': 0.667,
    'LA': 0.571,
    'SEA': 0.714,
    'GB': 0.857,
    'CAR': 0.667,
    'SF': 1,
    'NYG': 0.286,
    'DAL': 0.571,
    'NO': 0.857,
    'MIN': 0.714,
    'LAC': 0.286,
    'DEN': 0.286,
    'TB': 0.333,
    'MIA': 0
}

CONF = {
    'KC': 'AF',
    'NE':'AF',
    'NYJ': 'AF',
    'BUF': 'AF',
    'ATL': 'NF',
    'CHI': 'NF',
    'BLT': 'AF',
    'CIN': 'AF',
    'PIT': 'AF',
    'CLV': 'AF',
    'ARZ': 'NF',
    'DET': 'NF',
    'JAX': 'AF',
    'HST': 'AF',
    'OAK': 'AF',
    'TEN': 'AF',
    'PHI': 'NF',
    'WAS': 'NF',
    'IND': 'AF',
    'LA': 'NF',
    'SEA': 'NF',
    'GB': 'NF',
    'CAR': 'NF',
    'SF': 'NF',
    'NYG': 'NF',
    'DAL': 'NF',
    'NO': 'NF',
    'MIN': 'NF',
    'LAC': 'AF',
    'DEN': 'AF',
    'TB': 'NF',
    'MIA': 'AF'
}

In [16]:
offcol = ['RB', 'DB', 'OL', 'WR', 'DL', 'TE', 'QB', 'LB']
dffcol = ['OL', 'DL', 'DB','LB']

off = pd.DataFrame(columns=offcol)
for i in off.columns:
    off[i] = [0]*len(train)
    
dff = pd.DataFrame(columns=dffcol)
for i in dff.columns:
    dff[i] = [0]*len(train)

In [17]:
def low(a):
    return a.lower()

In [18]:
def team(a):
    map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
    if a in map_abbr.keys():
        return map_abbr[a]
    else:
        return a


In [19]:
def stadium_map(a):
    stadium = {
    'Gillette Stadium': ['Gillette Stadium'],
    'New Era Field': '',
    'Soldier Field': '',
    'Paul Brown Stadium': 'Paul Brown Stdium',
    'First Energy': ['First Energy Stadium', 'FirstEnergy Stadium', 'FirstEnergyStadium','FirstEnergy' ],
    'Ford Field': '', 
    'NRG Stadium': 'NRG',
    'Nissan Stadium': '',
    'Fedex Field': 'FedexField',
    'LA Memorial': ['Los Angeles Memorial Coliseum', 'Los Angeles Memorial Coliesum'],
    'Lambeau Field': 'Lambeau field',
    'Levis Stadium': '',
    'AT&T Stadium': '',
    'U.S. Bank Stadium': '',
    'M&T Bank Stadium': ['M&T Stadium', 'M & T Bank Stadium'],
    'BoA Stadium': ['Bank of America Stadium'],
    'Lucas Oil Stadium': '',
    'Arrowhead Stadium': '',
    'Mercedes-Benz Dome': ['Mercedes-Benz Superdome', 'Mercedes-Benz Dome', 'Mercedes-Benz Stadium'],
    'Heinz Field': '',
    'Raymond James Stadium': '',
    'StubHub Center': '',
    'Oakland-Alameda County Coliseum': 'Oakland Alameda-County Coliseum',
    'CenturyLink Field': ['CenturyLink', 'CenturyField'],
    'MetLife Stadium': ['MetLife', 'Metlife Stadium'],
    'Wembley Stadium': '',
    'Lincoln Financial Field': '',
    'Hard Rock Stadium': '',
    'State Farm Stadium': ['University of Phoenix Stadium'],
    'TIAA Bank Field': ['Everbank Field', 'EverBank Field'],
    'Estadio Azteca': '',
    'Empower Field': ['Broncos Stadium At Mile High', 'Broncos Stadium at Mile High', 'Sports Authority MA', 'Sports Authority Field at Mile High'],
    'Twickenham': 'Twickenham Stadium'
    }
    for i in range(len(stadium)):
        if a in stadium[list(stadium.keys())[i]]:
            return list(stadium.keys())[i]
            break
    else:
        return a


In [20]:
def turf_map(a):
    turf = {
    'Field Turf': ['FieldTurf', 'FieldTurf 360', 'FieldTurf360', 'Field turf'],
    'A_Turf': '',
    'Grass': ['grass'],
    'UBU': ['UBU Sports Speed S5-M', 'UBU Speed Series-S5-M'],
    'Artificial': ['Artifical'],
    'DD': ['DD GrassMaster'],
    'Natural': ['Natural Grass', 'Natural grass', 'Natural', 'Naturall Grass', 'natural grass'],
    'SIS': ['SISGrass'],
    'Twenty-Four/Seven Turf':'',
    }
    for i in range(len(turf)):
        if a in turf[list(turf.keys())[i]]:
            return list(turf.keys())[i]
    else:
        return a
    

In [21]:
def windcleaning(train):
    train['WindSpeed'] = train['WindSpeed'].replace(np.nan, '0').map(low)
    train['WindSpeed'] = train['WindSpeed'].replace([np.nan, 'calm', 'ssw', 'e', 'se'], '0')
    train['WindSpeed'] = train['WindSpeed'].replace('mph', '')
    train['WindSpeed'] = train['WindSpeed'].replace(' gusts up to ', '-')
    return train['WindSpeed']

def windspeed(a):
    if 'gusts up to' in a:
        a = a.replace('gusts up to', '-')
    elif 'mph' in a:
        a = a.replace('mph','')
    
    if '-' in a:
        x, y = a.split('-')
        a = (int(x)+int(y))/2
    

    try:
        return int(a)
    except:
        return a

In [22]:
def stdtype(a):
    a = a.lower()
    if ('ou' in a) or ('open' in a):
        return 'outdoor'
    elif ('in' in a) or ('close' in a) or ('dome' in a) or ('bowl' in a):
        return 'indoor'
    else:
        return a

In [25]:
# 시간단축가능?
def offset():
    global off
    for i in range(len(train)):
        temp = train['OffensePersonnel'][i].split(',')
        for j in range(len(temp)):
            x = temp[j].strip()
            off[x[2:4]][i] = int(x[0])
    
# def 

In [26]:
def age(a):
    res = 2019 - int(a[-4:])
    return res



In [27]:
def weather(a):
    weather = {
    'sun': 1,
    'cloud': 0,
    'clear': 1,
    'rain': -1,
    'snow': -1    
    }

    if ('sun' in a) or ('clear' in a) or ('fair' in a):
        return 1
    elif ('rain' in a) or ('snow' in a) or ('shower' in a):
        return -1
    elif ('cloud' in a) or ('coudy' in a) or ('clouid' in a):
        return 0.5
    elif ('cold' in a) or ('hazy' in a) or ('overcast' in a):
        return -0.5
    else:
        return 0

In [59]:
train = tt.copy()

In [60]:
import gc
gc.collect()

25

In [61]:
import datetime

In [62]:
train['PossessionTeam'] = train['PossessionTeam'].map(team)
train['HomeTeamAbbr'] = train['HomeTeamAbbr'].map(team)
train['VisitorTeamAbbr'] = train['VisitorTeamAbbr'].map(team)

train['OffenseFormation'] = train['OffenseFormation'].replace([np.nan, 'EMPTY'], 'NAN')
train['DefendersInTheBox'] = train['DefendersInTheBox'].replace(np.nan, 0)

train['PlayerHeight'] = train['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

train['Stadium'] = train['Stadium'].map(stadium_map)

train['Turf'] = train['Turf'].map(turf_map)

train['WindSpeed'] = windcleaning(train)
train['WindSpeed'] = train['WindSpeed'].map(windspeed)

train['StadiumType'] = train['StadiumType'].replace(np.nan, 'NAN').map(stdtype)

train['TotalScore'] = train['HomeScoreBeforePlay'] + train['VisitorScoreBeforePlay']

train['GameClock'] = int(train['GameClock'][0][:2])+float(train['GameClock'][0][3:5])/60
train['CumGameClock'] = train['GameClock'] / 15

h = [0] * len(train)
vt = [0] * len(train)
for i in range(len(train)):
    if train['Team'][i] == 'away':
        h[i] = train['VisitorTeamAbbr'][i]
        vt[i] = train['HomeTeamAbbr'][i]
    else:
        h[i] = train['HomeTeamAbbr'][i]
        vt[i] = train['VisitorTeamAbbr'][i]
train['VsTeam'] = vt
train['HomeTeam'] = h
        
offense = [0] * len(train)
for i in range(len(train)):
    if train['PossessionTeam'][i] == train['HomeTeam'][i]:
        offense[i] = 1    
train['IsOffense'] = offense

train['PlayerBirthDate'].map(age)

rish = [0]*len(train)
for i in range(len(train)):
    if train['NflId'][i] == train['NflIdRusher'][i]:
        rish[i] = 1
train['NflIdRusher'] = rish

train['GameWeather'] = train['GameWeather'].replace(np.nan, 'NAN').map(low).map(weather)

train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)

train['HomeField'] = train['FieldPosition'] == train['HomeTeam']
train['Field_eq_Possession'] = train['FieldPosition'] == train['PossessionTeam']

w_ratio = [0] * len(train)
for i in range(len(train)):
    if train['Season'][i] == 2017:
        w_ratio[i] = winratio17_18[train['HomeTeam'][i]]
    else:
        w_ratio[i] = winratio18_19[train['HomeTeam'][i]]
        
Vs_w_ratio = [0] * len(train)
for i in range(len(train)):
    if train['Season'][i] == 2017:
        Vs_w_ratio[i] = winratio17_18[train['VsTeam'][i]]
    else:
        Vs_w_ratio[i] = winratio18_19[train['VsTeam'][i]]
        
train['WinRatio'] = w_ratio
train['VsWinRatio'] = Vs_w_ratio
## 

In [64]:
len(train.columns)

59

In [65]:
a = pd.read_csv('input/FEd_train.csv')

In [66]:
a.head()

,X,Y,S,A,Dis,Orientation,Dir,NflId,JerseyNumber,Season,...,OFF_TE,OFF_QB,OFF_LB,DEF_OL,DEF_DL,DEF_DB,DEF_LB,WinRatio,VsWinRatio,Conf
0,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,29,2017,...,1,0,0,0,2,6,3,0.813,0,AF
1,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,97,2017,...,1,0,0,0,2,6,3,0.813,0,AF
2,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,50,2017,...,1,0,0,0,2,6,3,0.813,0,AF
3,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,56,2017,...,1,0,0,0,2,6,3,0.813,0,AF
4,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,38,2017,...,1,0,0,0,2,6,3,0.813,0,AF
